In [1]:
import re
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec
import random
import string
import math

# matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

use all calls from strelka: this is prefered since this takes into consideration of all soamtic calls:snvs and indels, high/moderate/low/modifier impacts.

in most papers, they only report mutation in coding regions including splicing site, excluding non-coding regions, i will get these numbers as well. synonymous/non-synonymous mutation are only in coding regions.

a good post about match

https://stackoverflow.com/questions/24958358/use-regular-expression-in-python-to-find-two-strings-in-line

a table details the equivalence of ontology an classic effect description

http://snpeff.sourceforge.net/SnpEff_manual.html

# 1. parsing strelka vcf files

In [2]:
def parse_strelka_vcf(vcf):
    df = pd.read_csv(vcf, comment='#', sep='\t', header=None, low_memory=False)
    patient = vcf.split('/')[4]
    df = df[[0,1,3,4,7]]
    df.columns = ['chr', 'pos', 'ref', 'alt', 'effect']
    df = df[(df['effect'].str.contains("HIGH"))|(df['effect'].str.contains("MODERATE"))|(df['effect'].str.contains("LOW"))]
    if not df.empty:
        df['impact'], df['impact_type'], df['gene'] = df['effect'].apply(lambda x: parse_effect(x)).str.split('@').str
        df['patient'] = patient
    df = df.drop('effect', axis=1)
    return df

In [3]:
# keep it easy for now pick HIGH, MODERATE and then LOW
def parse_effect(line):
    effs = line.split('EFF=')[1].split(',')
    #     extract impact, impact_type and gene
    effs = ['@'.join(list(np.array(re.split('\(|\|',ef))[[0,1,6]])) for ef in effs if ('HIGH' in  ef) or ('MODERATE' in ef) or ('LOW' in ef)]
    effs = list(set(effs))
    high = [ef for ef in effs if 'HIGH' in ef]
    moderate = [ef for ef in effs if 'MODERATE' in ef]
    low = [ef for ef in effs if 'LOW' in ef]
    if high:
        anno = high[0]
    elif moderate:
        anno = moderate[0]
    elif low:
        anno = low[0]
    else:
        print('ERROR!')
#     make sure the genes have the same name
    genes = [ef.split('@')[2] for ef in effs]
    
    return anno


In [4]:
# # example run
# ft = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02020/hg19a/GSH/A37247_A56468/strelka/23223/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf'
# # assert count_coding_mutations(ft) == 266
# df = parse_strelka_vcf(ft)

# 2. Scope: genes and cohort size

In [6]:
# 118 patients
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
with open(f) as file:
    patients = [line.strip() for line in file]
assert len(patients) == 118

In [7]:
# 12 SMGS
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed.txt'
with open (f2) as file:
    genes = [line.strip() for line in file]
assert len(genes) == 12

# 3. Strelka vcf files

In [8]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/124_patients_bam_vcf.txt'
df = pd.read_csv(f1, sep='\t', index_col='patient')
df.head(2)

,HIV_status,DNA_lib,source,status,RNA_lib,ribodepleted_lib,DNA_bam,RNA_bam,DNA_single_vcf,paired_mpileup_vcf,...,other_vcf,DNA_tc,RNA_tc,cnv,bbt_transcriptome,bbt_genome,bbt_transcriptome_other_bacterial,bbt_genome_other_bacterial,bbt_transcriptome_other_viral,bbt_genome_other_viral
patient,,,,,,,,,,,,,,,,,,,,,
HTMCP-03-06-02001,Positive,A37234,HTMCP_124,Malignant,A37700,A56295,/projects/analysis/analysis22/A37234/merge_bwa...,/projects/analysis/analysis22/IX3433/C67GDANXX...,/projects/analysis/analysis22/A37234/merge_bwa...,NaN,...,NaN,55.0,55.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...
HTMCP-03-06-02002,Negative,A37235,HTMCP_125,Malignant,A37701,A56296,/projects/analysis/analysis22/A37235/merge_bwa...,/projects/analysis/analysis22/IX3432/C67GDANXX...,/projects/analysis/analysis22/A37235/merge_bwa...,NaN,...,NaN,70.0,70.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...


In [9]:
ndf = df[['strelka_snv_vcf', 'strelka_indel_vcf']].dropna()
ndf.head(2)

,strelka_snv_vcf,strelka_indel_vcf
patient,,
HTMCP-03-06-02001,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02002,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [13]:
ndf = ndf.reindex(patients)

In [14]:
ndf.shape

(118, 2)

# 4. concatenate snvs and indels 

In [15]:
dfmg = pd.DataFrame()
for ix, row in ndf.iterrows():
    patient = ix
    snv_vcf = row['strelka_snv_vcf']
    indel_vcf = row['strelka_indel_vcf']
    snv_df = parse_strelka_vcf(snv_vcf)
    indel_df = parse_strelka_vcf(indel_vcf)
    mdf = pd.concat([snv_df, indel_df], sort=False)
    mdf = mdf.drop_duplicates(keep='first')

    if dfmg.empty:
        dfmg = mdf
    else:
        dfmg = pd.concat([dfmg, mdf], sort=False)


In [16]:
dfmg.head()
dfmg.shape

,chr,pos,ref,alt,impact,impact_type,gene,patient
39,1,2706109,C,G,SYNONYMOUS_CODING,LOW,TTC34,HTMCP-03-06-02001
58,1,4771973,C,T,NON_SYNONYMOUS_CODING,MODERATE,AJAP1,HTMCP-03-06-02001
78,1,6146051,G,C,NON_SYNONYMOUS_CODING,MODERATE,KCNAB2,HTMCP-03-06-02001
83,1,6579534,C,T,NON_SYNONYMOUS_CODING,MODERATE,PLEKHG5,HTMCP-03-06-02001
145,1,12170207,C,G,NON_SYNONYMOUS_CODING,MODERATE,TNFRSF8,HTMCP-03-06-02001


(33187, 8)

In [94]:
# subset 12 genes of interest
dfs = dfmg[dfmg.gene.isin(genes)&dfmg.patient.isin(patients)]
dfs.head(2)
dfs.shape

,chr,pos,ref,alt,impact,impact_type,gene,patient
5769,3,178936091,G,A,NON_SYNONYMOUS_CODING,MODERATE,PIK3CA,HTMCP-03-06-02001
5771,3,178952117,A,T,NON_SYNONYMOUS_CODING,MODERATE,PIK3CA,HTMCP-03-06-02001


(168, 8)

In [95]:
dfs = dfs[['gene', 'impact', 'patient']]
dfs.shape

dfs['impact_new'] = dfs['impact'].apply(lambda x: x.upper())
dfs.drop('impact', axis=1, inplace=True)
dfs.drop_duplicates(keep='first', inplace=True)
dfs.shape

(168, 3)

(148, 3)

In [96]:
dfs['impact_new'].unique()

array(['NON_SYNONYMOUS_CODING', 'FRAME_SHIFT', 'STOP_GAINED',
       'MISSENSE_VARIANT', 'SYNONYMOUS_VARIANT', 'SYNONYMOUS_CODING',
       'SPLICE_SITE_ACCEPTOR', 'START_GAINED', 'CODON_DELETION',
       'CODON_CHANGE_PLUS_CODON_DELETION'], dtype=object)

In [97]:
dfs = dfs.drop_duplicates().groupby(['gene', 'patient']).agg({'impact_new': ','.join}).reset_index()
dfs.head()

,gene,patient,impact_new
0,CASP8,HTMCP-03-06-02001,FRAME_SHIFT
1,CASP8,HTMCP-03-06-02012,STOP_GAINED
2,CASP8,HTMCP-03-06-02036,NON_SYNONYMOUS_CODING
3,CASP8,HTMCP-03-06-02239,MISSENSE_VARIANT
4,CASP8,HTMCP-03-06-02260,FRAME_SHIFT


In [98]:
def impact_type(x):
#     print(x)
    xsplit = list(set(x.split(',')))
    type = xsplit[0].upper()
    if len(xsplit) > 1:
        impact = 'Multiple'
    elif len(xsplit) == 1:
         if '+' in xsplit[0]:
             impact = 'Multiple'
         elif type == 'MISSENSE_VARIANT' or type == 'NON_SYNONYMOUS_CODING':
             impact = 'Non-synonymous'
         elif type == 'SYNONYMOUS_VARIANT' or type == 'SYNONYMOUS_CODING':
             impact = 'Synonymous'
         elif  type == 'SPLICE_SITE_ACCEPTOR' or  type == 'SPLICE_SITE_DONOR':
             impact = 'Splite site'
         elif type == 'STOP_LOST':
             impact = 'Stop lost'
         elif type == 'STOP_GAINED':
             impact = 'Stop gained'
         elif  type == 'START_LOST':
             impact = 'Start lost'
         elif type == 'START_GAINED':
             impact = 'Start gained'
         elif type == 'FRAME_SHIFT' or type == 'FRAMESHIFT_VARIANT':
             impact = 'Frameshift'
         elif type == 'INFRAME_DELETION' or type == 'CODON_INSERTION' or type == 'CODON_DELETION' or type == 'CODON_CHANGE_PLUS_CODON_INSERTION' or type == 'CODON_CHANGE_PLUS_CODON_DELETION' or type.lower() == 'disruptive_inframe_deletion':
             impact = 'Codon indel'                
         else: exit(1);print('ERROR');print(x)
    return impact

In [99]:
dfs['impact'] = dfs['impact_new'].apply(lambda x: impact_type(x))
dfs = dfs[['gene', 'patient', 'impact']].drop_duplicates()
dfs = dfs.set_index(['gene', 'patient'])['impact'].unstack()
dfs.head(2)
dfs.shape

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,HTMCP-03-06-02047,...,HTMCP-03-06-02344,HTMCP-03-06-02346,HTMCP-03-06-02354,HTMCP-03-06-02392,HTMCP-03-06-02411,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02435,HTMCP-03-06-02441,HTMCP-03-06-02447
gene,,,,,,,,,,,,,,,,,,,,,
CASP8,Frameshift,NaN,NaN,Stop gained,NaN,NaN,Non-synonymous,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAT1,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,Synonymous,NaN,NaN,...,NaN,NaN,Stop gained,NaN,NaN,Frameshift,Multiple,Non-synonymous,NaN,NaN


(12, 80)

# 5. add in covariates

In [100]:
# get meta track for example histology
f3 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
ddf = pd.read_csv(f3, sep='\t')
ddf = ddf[['patient', 'HIV_status', 'Putative_histology']]
edf = ddf.set_index('patient').T
edf.loc['Putative_histology',].unique()
edf = edf[patients]
edf.head(2)
edf.shape

array(['Squamous', 'Adenosquamous', 'Adeno', 'Neuroendocrine'],
      dtype=object)

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous


(2, 118)

In [101]:
dfs = pd.concat([edf, dfs], sort=False)

In [102]:
dfs.shape

(14, 118)

In [103]:
dfs = dfs[patients]

# 6. add in mutation rate (fraction of patients)

In [104]:
dfs['occurrence'] = (dfs.notnull().sum(axis=1)/(dfs.shape[1] -2))
dfs['percentage'] = ['{0}({1}%)'.format(i[0], int(round(i[1]*100))) for i in zip(dfs.index.tolist(), dfs.occurrence)]
# dfs.loc['HIV_status', 'percentage'] = 'HIV_Status'
# dfs.loc['Putative_histology', 'percentage' ] = 'Puatative_histology'
sdf = dfs.drop('occurrence', axis=1).set_index('percentage', drop=True)

In [105]:
sdf

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
percentage,,,,,,,,,,,,,,,,,,,,,
HIV_status(102%),Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology(102%),Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous
CASP8(7%),Frameshift,NaN,NaN,NaN,NaN,Stop gained,NaN,NaN,Non-synonymous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAT1(19%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,NaN,...,NaN,NaN,Frameshift,Multiple,NaN,Non-synonymous,NaN,NaN,NaN,NaN
FBXW7(10%),NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Non-synonymous,NaN,Frameshift,NaN,NaN,NaN,NaN
MAPK1(5%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN
MLL2(15%),NaN,Stop gained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Frameshift,NaN,NaN,NaN,NaN
PCDHA9(3%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCDHGA12(5%),NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN


In [106]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed_details.txt'
sdf.to_csv(of, sep='\t')